<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/1D_CNN_IMDB_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
vocab_size=10000
(X_train, y_train), (X_test, y_test)=datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [6]:
max_len=200
X_train=pad_sequences(X_train, maxlen=max_len)
X_test=pad_sequences(X_test, maxlen=max_len)

In [9]:
print(X_train.shape)
print(X_test.shape)

(25000, 200)
(25000, 200)


In [11]:
y_train[:5]

array([1, 0, 0, 1, 0])

In [12]:
from tensorflow.keras.layers import Embedding, Dense, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim=256
dropout_ratio=0.3
kernel_size=3
num_filters=256
hidden_units=128

model=Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))#num_filters는 커널(가중치)의 개수, kerner_size는 커널의 높이(크기)
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es=EarlyStopping(monitor='val_loss', mode='min', patience=3, verbose=1)
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=20, callbacks=[es, mc], validation_data=(X_test, y_test))

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.4018 - acc: 0.8046
Epoch 1: val_acc improved from -inf to 0.88676, saving model to best_model.h5
782/782 [==============================] - 168s 213ms/step - loss: 0.4018 - acc: 0.8046 - val_loss: 0.2695 - val_acc: 0.8868
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.2027 - acc: 0.9216
Epoch 2: val_acc improved from 0.88676 to 0.89180, saving model to best_model.h5
782/782 [==============================] - 144s 184ms/step - loss: 0.2027 - acc: 0.9216 - val_loss: 0.2628 - val_acc: 0.8918
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.0973 - acc: 0.9665
Epoch 3: val_acc did not improve from 0.89180
782/782 [==============================] - 143s 183ms/step - loss: 0.0973 - acc: 0.9665 - val_loss: 0.3094 - val_acc: 0.8849
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.0421 - acc: 0.9854
Epoch 4: val_acc did not improve from 0.89180
782/782 [=

In [14]:
loaded_model=load_model('best_model.h5')
print("\n 테스트 정확도 : %.4f"%(loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 27s 34ms/step - loss: 0.2628 - acc: 0.8918

 테스트 정확도 : 0.8918
